In [ ]:
import numpy as np
import pandas as pd

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import initializers

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from scipy.optimize import curve_fit


import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from IPython.display import Image
import plotly.io as pio

import time
import gc
import os
import random
import matplotlib.pyplot as plt
import math

In [2]:
data = pd.read_csv('/home/egracheva/Work/PolyInfo/DFT properties/LTE_DFT.csv', sep='\t')

In [3]:
# Removing the outlier at HOMO=-10
data = data[data['HOMO']>-10]
# Dropping all the correlated features
data = data.drop(['Entropy', 'ZeroPointEn', 'SumEnthal', 'SumFreeEner', 'SumEner'], axis = 1)

In [4]:
x = data.iloc[:, 2:-1].values
y = data['Linear expansion coefficient'].values.reshape(-1, 1)

In [5]:
sc_x = StandardScaler()
x_scaled = sc_x.fit_transform(x)
sc_y = StandardScaler()
y_scaled = sc_y.fit_transform(y)

In [6]:
n_components = x_scaled.shape[1]

In [7]:
seed = 123

In [8]:
def model_notrain(n_units1, n_units2, n_units3, weight):
    
    model = Sequential()
    model.add(Dense(int(n_units1), 
                    input_dim=int(n_components),
                    kernel_initializer=initializers.constant(value=weight), 
                    activation='relu'))        
    if n_units2 != 0:
        model.add(Dense(int(n_units2), 
                        kernel_initializer=initializers.constant(value=weight), 
                        activation='relu'))
    if n_units3 != 0:
        model.add(Dense(int(n_units3), 
                        kernel_initializer=initializers.constant(value=weight), 
                        activation='relu'))
    model.add(Dense(1, kernel_initializer='glorot_normal'))
    # Compile model
    model.compile(loss=root_mean_squared_error, optimizer='sgd')
    
    return model

In [9]:
def model_train(learning_rate, n_units1, n_units2, n_units3, weight):
    
    model = Sequential()
    model.add(Dense(int(n_units1), 
                    input_dim=int(n_components), 
                    kernel_initializer=initializers.random_normal(mean=weight, stddev=0.05), 
                    activation='relu'))        
    if n_units2 != 0:
        model.add(Dense(int(n_units2), 
                        kernel_initializer=initializers.random_normal(mean=weight, stddev=0.05), 
                        activation='relu'))
    if n_units3 != 0:
        model.add(Dense(int(n_units3), 
                        kernel_initializer=initializers.random_normal(mean=weight, stddev=0.05), 
                        activation='relu'))
    model.add(Dense(1, kernel_initializer='glorot_normal'))
    # Compile the model
    my_adam = Adam(lr=math.pow(10,-float(learning_rate)))
    model.compile(loss=root_mean_squared_error, optimizer=my_adam)
    
    return model

In [10]:
def random_split(x, y, random_state=123):

    full_idx = np.array([x for x in range(x.shape[0])])
    train_idx = np.random.choice(full_idx, 
                                 size=math.ceil(0.8*x.shape[0]), 
                                 replace = False)
    x_train = x[train_idx]
    y_train = y[train_idx].reshape(-1, 1)
    
    valid_idx = full_idx[np.isin(full_idx, train_idx, invert=True)]
    x_valid = x[valid_idx]
    y_valid = y[valid_idx].reshape(-1, 1)
    
    return x_train, x_valid, y_train, y_valid

In [37]:
kfold_splits = 10 # 17
n_runs = 4
best_to_keep = 20
rmse_threshold = 5
min_kept = 5
n_geoms = 3
comment = "const.search+rand.fix.train"

In [38]:
# learning_rates = [float(ialpha/10.) for ialpha in range(20,40,2)]
# batch_sizes = [4, 8, 16, 32, 64, 128]
# n_units1_bounds = [2, 4, 8, 12, 16, 24, 32, 64]
# n_units2_bounds = [2, 4, 8, 12, 16, 24, 32, 64]
# n_units3_bounds = [2, 4, 8, 12, 16, 24, 32, 64]
# weight_range = [-0.1, -0.08, -0.06, -0.04, -0.02, 0.02, 0.04, 0.06, 0.08, 0.1]
learning_rates = [float(ialpha/10.) for ialpha in range(20,40,10)]
batch_sizes = [16, 32]
n_units1_bounds = [2, 4, 8]
n_units2_bounds = [2, 4, 8]
n_units3_bounds = [2, 4, 8]
weight_range = [-0.06, 0.06]

In [14]:
settings = {"Runs":n_runs,
            "Number of splits":kfold_splits,
            "Phase 1 models kept":best_to_keep,
            "RMSE threshold":rmse_threshold,
            "Phase 2 minimum outcome":min_kept,
            "Learning rates":learning_rates,
            "First layer":n_units1_bounds,
            "Second layer":n_units2_bounds,
            "Third layer":n_units3_bounds,
            "Comment":comment
          }

In [15]:
# detect the current working directory and print it
path = os.getcwd()  

suffix = "Kept{}_RMSEthr{}_Min{}{}".format(best_to_keep, rmse_threshold, min_kept, comment)
output_dir = "{}/NN/ENAS/{}".format(path, suffix)

try:  
    os.mkdir(output_dir)
except OSError:  
    print ("Creation of the directory %s failed" % output_dir)
else:  
    print ("The directory %s is created" % output_dir)

Creation of the directory /home/egracheva/Work/PolyInfo/DFT properties/NN/ENAS/Kept20_RMSEthr5_Min5const.search+rand.fix.train failed


In [16]:
import json
with open('{}/Settings.json'.format(output_dir), 'w') as json_file:  
    json.dump(settings, json_file)

In [17]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [18]:
def exponenial_func(x, a, b, c):
    return a*np.exp(-b*x)+c

In [19]:
# Here, I search for the best geometry without training the network, jsut passing the data throu it and seeing the result
def geometry_search(x, y, n_units1_bounds, n_units2_bounds, n_units3_bounds):    
    
    columns = ['n_units1', 'n_units2', 'n_units3', 'mean_rmse', 'best_weight']
    geoms = pd.DataFrame()
    
    for n_units1 in n_units2_bounds:
        for n_units2 in n_units2_bounds:
            for n_units3 in n_units3_bounds:
                #print("The neural network parameters are: [{}, {}, {}]".format(n_units1, n_units2, n_units3))
                rmse_scores = []
                for weight in weight_range:
                    K.clear_session()
                    # Create a new model
                    model = model_notrain(n_units1, n_units2, n_units3, weight)
                    # Evaluate without training
                    rmse = model.evaluate(x, y, verbose=0)
                    rmse_scores.append(rmse)
                    #print(weight, rmse)
                mean_rmse = np.mean(rmse)
                best_weight = weight_range[np.argmin(rmse)]
                geoms = geoms.append([[n_units1, n_units2, n_units3, mean_rmse, best_weight]], ignore_index=True)
    geoms.columns = columns
    best_geoms = geoms.sort_values(by="mean_rmse").iloc[:n_geoms, :].reset_index(drop=True)
    
    return best_geoms

In [40]:
def each_fold_optimization(x, y, learning_rate, batch_size, n_units1, n_units2, n_units3, weight):

    print("The neural network parameters are: [{}, {}]".format(learning_rate, batch_size))
    
    
    full_idx = np.array([x for x in range(x.shape[0])])
    train_idx = np.random.choice(full_idx,
                                 size=math.ceil(0.8*x.shape[0]),
                                 replace = False)
    x_train = x[train_idx]
    y_train = y[train_idx].reshape(-1, 1)
    
    valid_idx = full_idx[np.isin(full_idx, train_idx, invert=True)]
    x_valid = x[valid_idx]
    y_valid = y[valid_idx].reshape(-1, 1)
    
    x_train, x_valid, y_train, y_valid =random_split(x, y, seed)
    
    pred_valid_bag = np.zeros((x_valid.shape[0], n_runs))
    epochs = np.array([1, 5, 10, 15, 20])
    valid_evol = []
    for epoch in epochs:
        for i in range(n_runs):
            K.clear_session()
            # Create a new model
            model_opt = model_train(learning_rate, n_units1, n_units2, n_units3, weight)
            # Callbacks
            model_opt.fit(x_train,
                          y_train,
                          batch_size = batch_size,
                          validation_data = (x_valid, y_valid),
                          epochs=epoch,
                          verbose=0)

            # Prediction
            pred_valid_bag[:,i] = model_opt.predict(x_valid).ravel()
        pred_valid_mean = np.mean(pred_valid_bag, axis = 1)
        rmse_valid_epoch = np.sqrt(mean_squared_error(y_valid, pred_valid_mean))
        valid_evol.append(rmse_valid_epoch)
    valid_evol = np.array(valid_evol)
    try:
        popt, _ = curve_fit(exponenial_func, epochs, valid_evol)
        rmse = np.sqrt(mean_squared_error(valid_evol, exponenial_func(epochs, *popt)))
    # In case the exponential failed to fit
    except:
        print("Could not fit the exponential function to the data")
        rmse = np.inf
    
    score = np.log(rmse) - popt[0]
    print(valid_evol)
    print("RMSE: {0}".format(rmse))
    print("A: {0}".format(popt[0]))
    print("SCORE:")
    print(score)
    
    return score

In [41]:
K.clear_session()

In [42]:
opt_step = 0
n_images = x_scaled.shape[0]
kfold = KFold(kfold_splits, shuffle = True)
fold = 0

folds = np.array(range(0, kfold_splits)) # The folds of the interest

best_confs = []

for train, test in kfold.split(x_scaled):
    if fold in folds:
        print("********************************")
        print("Started fold #{}...".format(fold))
        
        x_train_val = x_scaled[train,:]
        y_train_val = y_scaled[train,:]    
        
        # Grid search
        best_score = 100 # Some large initial value
        
        print("Geometry search starting...\n")
        best_geoms = geometry_search(x_train_val, y_train_val, n_units1_bounds, n_units2_bounds, n_units3_bounds)
        print("Best {} geometries:".format(n_geoms))
        print(best_geoms)
        
        print("********************************")
        print("Architecture search starting...\n")
        
        for _, geom in best_geoms.iterrows():
            n_units1, n_units2, n_units3, _, weight = geom
            print("Current conf:")
            print("Number of units in the 1st layer: {}".format(n_units1))
            print("Number of units in the 2nd layer: {}".format(n_units2))
            print("Number of units in the 3rd layer: {}".format(n_units3))
            print("Best weight: {}".format(weight))
            for learning_rate in learning_rates:
                for batch_size in batch_sizes:                    
                    score = each_fold_optimization(x = x_train_val, 
                                                   y = y_train_val, 
                                                   learning_rate = learning_rate, 
                                                   batch_size = batch_size, 
                                                   n_units1 = n_units1, 
                                                   n_units2 = n_units2, 
                                                   n_units3 = n_units3, 
                                                   weight = weight)
                    if score < best_score: 
                        best_score = score
                        best_conf = [n_units1, n_units2, n_units3, learning_rate, batch_size]
                        
        print("Best score: " + str(best_score))
        print("Best configuration: " + str(best_conf))

        best_confs.append(best_conf)
        pd.DataFrame(best_confs).to_csv("{}/Best_configurations.csv".format(output_dir))
        
        del x_train_val
        del y_train_val
        gc.collect()
        
        print("Finished the " + str(fold) + " fold")
        print("----------------------------------------------------")
        fold += 1
    else:   
        print("Skipped the " + str(fold) + " fold")
        print("----------------------------------------------------")  
        fold += 1
        best_confs.append([0,0,0,0,0])
        pd.DataFrame(best_confs).to_csv("{}/Best_configurations.csv".format(output_dir))    

********************************
Started fold #0...
Geometry search starting...

Best 3 geometries:
   n_units1  n_units2  n_units3  mean_rmse  best_weight
0         8         8         2   0.806223        -0.06
1         8         8         8   0.807206        -0.06
2         4         8         8   0.807317        -0.06
********************************
Architecture search starting...

Current conf:
Number of units in the 1st layer: 8.0
Number of units in the 2nd layer: 8.0
Number of units in the 3rd layer: 2.0
Best weight: -0.06
The neural network parameters are: [2.0, 16]
[1.09211039 1.10405465 1.05463261 1.0998508  1.10242385]
RMSE: 0.01843661537311415
A: 0.1715890027777932
SCORE:
-4.165005628574257
The neural network parameters are: [2.0, 32]
[1.12240546 1.09387138 1.06793649 1.123052   1.12281464]
RMSE: 0.020507590310171166
A: 2.895803197076994
SCORE:
-6.78276339941099
The neural network parameters are: [3.0, 16]
[1.22961245 1.22880271 1.22570837 1.22838695 1.22836061]
RMSE: 0.00

UnboundLocalError: local variable 'popt' referenced before assignment

In [ ]:
best_confs

In [ ]:
def each_fold_prediction(x_train, x_valid, learning_rate, n_units1, n_units2, n_units3, batch_size):
    
    prediction_train_bag = np.zeros((x_train.shape[0], n_runs))
    prediction_valid_bag = np.zeros((x_valid.shape[0], n_runs))
    #prediction_test_bag = np.zeros((x_test.shape[0], n_runs))
    
    rmse_train = np.zeros((n_runs, 1))
    rmse_valid = np.zeros((n_runs, 1))
    score = np.zeros((n_runs, 1))
    
    print("Running for the prediction...")
    for i in range(n_runs):
        if i%1==0:
            print(str(i) + " runs are done")
        
        model_final = model_train(learning_rate, n_units1, n_units2, n_units3, weight)
        model_final.fit(x_train, 
                        y_train, 
                        batch_size = batch_size, 
                        validation_data = (x_valid, y_valid), 
                        epochs=150, 
                        verbose = 0)
        # Prediction
        prediction_train_bag[:,i] = model_final.predict(x_train).ravel()
        prediction_valid_bag[:,i] = model_final.predict(x_valid).ravel()
        #prediction_test_bag[:,i] = model_final.predict(x_test).ravel()
                
        rmse_train[i] = np.sqrt(mean_squared_error(y_train, prediction_train_bag[:, i]))
        rmse_valid[i] = np.sqrt(mean_squared_error(y_valid, prediction_valid_bag[:, i]))
        score[i] = rmse_valid[i]/rmse_train[i]
        
        # Cleaning up
        ! rm -rf /tmp/weights_int.hdf5
        K.clear_session()
        del model_final
    
    # The outputs are all scaled (need to be unscaled)
    return prediction_train_bag, prediction_val_bag, score

In [ ]:
opt_step = 0
n_images = X_data_scaled_pca.shape[0]
kfold = KFold(kfold_splits, shuffle = False)
fold = 0
folds = np.array(range(0,kfold_splits)) # The folds of the interest

for train_valid, test in kfold.split(X_data_scaled_pca):
    if fold in folds:
        print("Started the " + str(fold) + " fold...")
        
        ceiling = int(0.9*len(train_val))
        train = train_valid[:ceiling]
        valid = train_valid[ceiling:]
        
        x_train, x_valid = x_scaled[train,:], x_scaled[valid,:]
        y_train, y_valid = y_scaled[train,:], y_scaled[valid,:]
        
        n_units1, n_units2, n_units3, learning_rate, batch_size = best_confs[fold]
        prediction_train_bag, prediction_val_bag, score = \
                              each_fold_prediction(x_train, x_valid, learning_rate, n_units1, n_units2, n_units3, batch_size)

        train_predictions = prediction_train_bag[:, np.array(score<rmse_threshold).ravel()]
        valid_predictions = prediction_val_bag[:, np.array(score<rmse_threshold).ravel()]
        #test_predictions = prediction_test_bag[:, np.array(score<rmse_threshold).ravel()]

        #pd.DataFrame(test_predictions).to_csv("{}/Test_predictions_fold{}.csv".format(output_dir, fold), index = False, header = False)
        pd.DataFrame(valid_predictions).to_csv("{}/Validation_predictions_fold{}.csv".format(output_dir, fold), index = False, header = False)       
        pd.DataFrame(train_predictions).to_csv("{}/Train_predictions_fold{}.csv".format(output_dir, fold), index = False, header = False)
        
        mean_train_predictions = np.mean(train_predictions, axis=1)
        mean_valid_predictions = np.mean(valid_predictions, axis=1)
        sigma_train_predictions = np.std(train_predictions, axis=1)
        sigma_valid_predictions = np.std(valid_predictions, axis=1)
        
        rmse_train = mean_squared_error(y_train, mean_train_predictions)
        rmse_valid = mean_squared_error(y_valid, mean_valid_predictions)
        r2_train = r2_score(y_train, mean_train_predictions)
        r2_valid = r2_score(y_valid, mean_valid_predictions)
        mae_train = mean_absolute_error(y_train, mean_train_predictions)
        mae_valid = mean_absolute_error(y_valid, mean_valid_predictions)
        
        print("Fold #{0}:".format(fold))
        print("RMSE train, valid: {0:0.4f}, {1:0.4f}".format(rmse_train, rmse_valid))
        print("MAE train, valid: {0:0.4f}, {1:0.4f}".format(mae_train, mae_valid))
        print("R2 score train, valid: {0:0.4f}, {1:0.4f}".format(r2_train, r2_valid))
        
        
        # ~~~~~~~~~~~~~~~~~~PLOTS~~~~~~~~~~~~~~~~~~~~
        plt.figure(figsize=(12, 8))
        # Setting plot limits
        y_true_min = min(np.min(y_train), np.min(y_valid))#, np.min(y_test))
        y_true_max = max(np.max(y_train), np.max(y_valid))#, np.max(y_test))
        y_pred_min = min(np.min(mean_train_predictions), np.min(mean_valid_predictions))#, np.min(y_test_pred))
        y_pred_max = max(np.max(mean_train_predictions), np.max(mean_valid_predictions))#, np.max(y_test_pred))

        axmin = y_true_min-0.1*(y_true_max-y_true_min)
        axmax = y_true_max+0.1*(y_true_max-y_true_min)
        aymin = y_pred_min-0.1*(y_pred_max-y_pred_min)
        aymax = y_pred_max+0.1*(y_pred_max-y_pred_min)

        plt.xlim(min(axmin, aymin), max(axmax, aymax))
        plt.ylim(min(axmin, aymin), max(axmax, aymax))
        
        plt.errorbar(y_train, 
                    mean_train_predictions,
                    yerr = sigma_train_predictions,
                    fmt='o',
                    label="Train",
                    elinewidth = 1, 
                    ms=7,
                    mfc='#7cb2cc',
                    markeredgewidth = 0,
                    alpha=0.7)
        plt.errorbar(y_valid, 
                    mean_valid_predictions,
                    yerr = sigma_valid_predictions,
                    elinewidth = 1,
                    fmt='o',
                    label="Validation", 
                    ms=7, 
                    mfc='#f4a582',
                    markeredgewidth = 0,
                    alpha=0.7)
        # Plot X=Y line
        plt.plot([max(plt.xlim()[0], plt.ylim()[0]), 
                  min(plt.xlim()[1], plt.ylim()[1])],
                 [max(plt.xlim()[0], plt.ylim()[0]), 
                  min(plt.xlim()[1], plt.ylim()[1])],
                 ':', color = '#595f69')
        
        plt.xlabel('Observations LTE', fontsize = 12)
        plt.ylabel('Predictions LTE', fontsize = 12)
        plt.legend()
        # ~~~~~~~~~~~~~~~~~~CHANGED HERE~~~~~~~~~~~~~~~~~~~~
        # Added fold number
        plt.savefig("{}/Plot_predictions_fold{}.png".format(output_dir, fold)', bbox_inches='tight', dpi=80)
        plt.close()
        
        del x_train, x_valid
        del y_train, y_valid
        gc.collect()
        
        print("Finished the " + str(fold) + " fold")
        print("----------------------------------------------------")
        fold += 1
    else:   
        print("Skipped the " + str(fold) + " fold")
        print("----------------------------------------------------")  
        fold += 1        

In [ ]:
true = [0, 3, 6, 34, 2, 77, 14, 47]
pred = [2, 6, 7, 38, 4, 64, 13, 53]

In [ ]:
sigma = [0.1, 0.2, 0.2, 10, 0.5, 5, 2, 10]

In [ ]:
np.sqrt(np.sum())

In [ ]:
ceiling

In [ ]:
train = train_val[:ceiling]
val = train_val[ceiling:]

In [ ]:
train_data = data[:50]
test_data = data[50:]

In [ ]:
final_prediction_test = np.zeros((n_images, 1))
final_std_test = np.zeros((n_images, 1))

In [ ]:
for f in range(17):
    print(f)
    test_fold = pd.read_csv("./Test_predictions_fold{}_PCA20_2step_5min_rmseratio5_yields5.csv".format(f), sep=",", header = None).values 

    # Getting rid of first zeros
    #test_fold = test_fold[:,1:50]
    
    final_prediction_test[f*5:f*5+5] = np.mean(sc_y.inverse_transform(test_fold), axis = 1).reshape((5,1))
    final_std_test[f*5:f*5+5] = np.std(sc_y.inverse_transform(test_fold), axis = 1).reshape((5,1))

In [ ]:
r2_score(Y_data, final_prediction_test)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
for ialpha in range(0,10,3):
    print(ialpha)
    print(ialpha/10.)
    b = ialpha/10.
    print(10.**-b)

In [ ]:
a = [10**(float(-1*ialpha/10)) for ialpha in range(10,20,1)]
b = [10**(float(-1*ialpha/10)) for ialpha in range(20,25,2)]

train = []
valid = []
score = []
color = []
for i, a_i in enumerate(a):
    for j , a_j in enumerate(a):
        for n, b_n in enumerate(b):
            for m, b_m in enumerate(b):
                train.append(a_i)
                valid.append(a_j)
                color.append(b_m+b_n)
    #         if a_j/a_i>5 or a_j<a_i*0.8:
    #             score_i = 1
    #         else:
    #             score_i = 0
                score_i = np.log(np.abs(a_j-a_i)*(b_m+b_n))
                score.append(score_i)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot

myplot=go.Scatter3d(x=train, 
                    y=valid, 
                    z=score,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=color,
                        colorscale='Viridis',
                        opacity=0.8
                        )
                   )

fig = go.Figure(data=[myplot])

fig.layout.update(scene = dict(
                    xaxis_title='TRAIN',
                    yaxis_title='VALID',
                    zaxis_title='SCORE')
                 )

plot(fig, filename='myplot.html')

In [ ]:
data = []
trace = dict(
        name = name,
        x = train, y = valid, z = score,
        type = "scatter3d",    
        mode = 'markers',
        marker = dict( size=3, color=color, line=dict(width=0) ) )
data.append( trace )